### Pytorch

In [141]:
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import r2_score

import wandb

In [142]:
ARCHITECTURE = 'FFNN'
WANDB_API_KEY = '5fed3be7a2bb71354e65254cba73f68f903cc854'
CONF_INDEX = 13
CONF = {
    'lr': [0.001, 0.00001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001],
    'num_epochs': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100],
    'gamma': [0.05, 0.05, 0.5, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.25],
    'scheduler_step_size': [20, 20, 20, 50, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20],
    'batch_size': [32, 32, 32, 32, 128, 32, 32, 32, 32, 32, 32, 32, 32, 64],
    'run_name': [
        'Case 1',
        'Case 2',
        'Case 3',
        'Case 4',
        'Case 5',
        'Case 6',
        'Case 7',
        'Case 8',
        'Case 9',
        'Case 10',
        'Case 11',
        'Case 12',
        'Case 13',
        'Case 14',
    ],
    'nn_size': [
        'medium_wide',
        'medium_wide',
        'medium_wide',
        'medium_wide',
        'medium_wide',
        'small',
        'large',
        'medium_wide',
        'medium_wide',
        'medium_wide',
        'medium_wide',
        'medium_wide',
        'medium_wide',
        'large',
    ],
    'act_func': [
        'leaky_relu',
        'leaky_relu',
        'leaky_relu',
        'leaky_relu',
        'leaky_relu',
        'leaky_relu',
        'leaky_relu',
        'relu',
        'tanh',
        'sigmoid',
        'leaky_relu',
        'leaky_relu',
        'leaky_relu',
        'relu',
    ],
    'loss_func': [
        'mse',
        'mse',
        'mse',
        'mse',
        'mse',
        'mse',
        'mse',
        'mse',
        'mse',
        'mse',
        'hubert',
        'mse',
        'mse',
        'hubert',
    ],
    'optimizers': [
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'sgd',
        'rmsprop',
        'adam',
        'rmsprop',
    ]
}

act_functions = {
    'leaky_relu': tf.keras.layers.LeakyReLU(alpha=0.01),
    'relu': 'relu',
    'tanh': 'tanh',
    'sigmoid': 'sigmoid',
}

loss_functions = {
    'mse': 'mse',
    'hubert': 'huber', # risk
}

optimizers = {
    'sgd': tf.keras.optimizers.SGD(learning_rate=CONF['lr'][CONF_INDEX] / 10),
    'rmsprop': tf.keras.optimizers.RMSprop(learning_rate=CONF['lr'][CONF_INDEX] / 10),
    'adam': tf.keras.optimizers.Adam(learning_rate=CONF['lr'][CONF_INDEX] / 10),
}

NAME = CONF['run_name'][CONF_INDEX]
LEARNING_RATE = CONF['lr'][CONF_INDEX] / 10
NUM_EPOCHS = CONF['num_epochs'][CONF_INDEX]
GAMMA = CONF['gamma'][CONF_INDEX]
SCHEDULER_STEP_SIZE = CONF['scheduler_step_size'][CONF_INDEX]
BATCH_SIZE = CONF['batch_size'][CONF_INDEX]
NN_SIZE = CONF['nn_size'][CONF_INDEX]
ACT_FUNC = act_functions[CONF['act_func'][CONF_INDEX]]
LOSS_FUNC = loss_functions[CONF['loss_func'][CONF_INDEX]]
OPTIMIZER_FUNCT = optimizers[CONF['optimizers'][CONF_INDEX]]

In [143]:
wandb.login(key=WANDB_API_KEY)

wandb.init(
    project='nn_tensorflow',
    name=NAME,
    config={
        'dataset': 'wines',
        'architecture': ARCHITECTURE,
        'nn_size': NN_SIZE,
        'act_func': ACT_FUNC,
        'loss_func': LOSS_FUNC,
        'optimizer_func': OPTIMIZER_FUNCT,
        'LR': LEARNING_RATE,
        'gamma': GAMMA,
        'step_size': SCHEDULER_STEP_SIZE,
        'epochs': NUM_EPOCHS,
        'batch_size': BATCH_SIZE,
    }
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


accuracy_r2,▁▅▇█████████████████████████████████████
basic_accuracy,▁▂▆▇▇▇██████████████████████████████████
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy_r2,0.17324
basic_accuracy,0.51462
train_loss,0.59597
val_loss,0.63121


In [144]:
df = pd.read_csv('data/cleaned_data.csv', delimiter = ';')
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6,0
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6,0


In [145]:
# Split the dataset into features and target variable
X = df.drop('quality', axis=1)  # Features
y = df['quality']               # Target variable

# We use stratify on y so we get same wine quality distribution in test and train dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27, stratify = y)

print('Training set shape:', X_train.shape, y_train.shape)
print('Testing set shape:', X_test.shape, y_test.shape)

# transform data to correct type
X_train, X_test = X_train.values, X_test.values
y_train, y_test = y_train.values, y_test.values
print(type(X_train), type(y_train), type(X_test), type(y_test))

Training set shape: (5197, 12) (5197,)
Testing set shape: (1300, 12) (1300,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Neural network architecture

- Number of inputs for first layer is same as the number of attributes (columns) in our dataset
- Number of outputs in last layer needs to be equal to the number of values that network is expected to predict. In our case we are trying to predict wine quality (range <3,9>)
- The number of neurons (inputs and outputs) per layer in hidden layers can vary - we need to try different values to find better network performace. A good rule of thumb is to start with a smaller number of neurons and gradually increase the number of neurons until you find the optimal number that gives you the best performance.
- We choose **ReLU as actiovation function**. In general, it works well
- MSELoss (Mean squared error) is commonly use when you want to penalize larger errors more than smaller ones (common choise for regression problems)
- We choose **SGD optimizer** because we have small dataset and problem with small complexity 

In [146]:
# Define the model architectures
model = None
if NN_SIZE == 'small':
    model = keras.Sequential([
        tf.keras.layers.Dense(8, activation=ACT_FUNC, input_shape=(12,)),
        tf.keras.layers.Dense(4, activation=ACT_FUNC),
        tf.keras.layers.Dense(1)
    ])
elif NN_SIZE == 'medium_wide':
    model = keras.Sequential([
        tf.keras.layers.Dense(15, activation=ACT_FUNC, input_shape=(12,)),
        tf.keras.layers.Dense(33, activation=ACT_FUNC),
        tf.keras.layers.Dense(16, activation=ACT_FUNC),
        tf.keras.layers.Dense(13, activation=ACT_FUNC),
        tf.keras.layers.Dense(1)
    ])
elif NN_SIZE == 'large':
    model = keras.Sequential([
        tf.keras.layers.Dense(128, activation=ACT_FUNC, input_shape=(12,)),
        tf.keras.layers.Dense(54, activation=ACT_FUNC),
        tf.keras.layers.Dense(33, activation=ACT_FUNC),
        tf.keras.layers.Dense(72, activation=ACT_FUNC),
        tf.keras.layers.Dense(50, activation=ACT_FUNC),
        tf.keras.layers.Dense(110, activation=ACT_FUNC),
        tf.keras.layers.Dense(1)
    ])

In [147]:
class PredictCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
    
    def on_epoch_end(self, epoch, logs={}):
        if (epoch + 1) % SCHEDULER_STEP_SIZE == 0:
            old_lr = self.model.optimizer.lr.numpy()
            self.model.optimizer.learning_rate.assign(old_lr * GAMMA)
        y_pred_val =  self.model.predict(X_test).tolist()
        y_true_val =  self.y_test.tolist()
        accuracy_r2 = round(r2_score(y_true_val, y_pred_val), 5)
        hit, miss = 0, 0
        for index, _ in enumerate(y_pred_val):
            if round(y_pred_val[index][0]) == y_true_val[index]:
                hit += 1
            else:
                miss += 1
        basic_accuracy = hit / (hit + miss)
        wandb.log({
            'train_loss': logs['loss'],
            'val_loss': logs['val_loss'],
            'accuracy_r2': accuracy_r2,
            'basic_accuracy': basic_accuracy
        })

# Compile the model
model.compile(
    optimizer=OPTIMIZER_FUNCT,
    loss=LOSS_FUNC
)

history = model.fit(
    X_train,
    y_train,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    verbose=0,
    validation_data=(X_test, y_test),
    validation_freq=1,
    callbacks=[PredictCallback(X_test, y_test)]
)

41/41 [==============================] - 0s 611us/step


wandb: Network error (ConnectionError), entering retry loop.


### What to do in the next week
- Try 2 hidden layers
- Try different number of neurons in hidden layers
- Try different activation functions
- Use validation data for tunning pytorch NN


- validaciu musime volat v cykle po kazdej epoche, v cykle nastavovat net.val, net.train [X]
- len testovaciu preprocesingovat [X]
- learning rate treba dynamicky menit a dobre je to nastaviť na mensi [X]
- spravit dobry shuffel [X]
- wandb [X]
- config [X]
- accuracy riesit tak ze predikujeme napr 4.6, tak to zaokruhlime na 5 a pozrieme ci sme sa trafili povedal
- asi neriesiť outliers
- try other architecture (vacsi mode, sirsi model, inu aktivacnu funkciu)
- TensorFlow implementation